In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import scipy.spatial as ssp

In [4]:
def load():
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    words = words_a + words_p
    return words


In [7]:
def get_sentences(words):
    sentences_word  = []
    sent_word = []
    for index, word in enumerate(words):
        if word in ['?','.','!']:
            sent_word += [word]
            sentences_word += [sent_word]
            sent_word = []
        else:
            sent_word += [word]
    return sentences_word

In [146]:
def in_embed_form(batch, KB, dic):
    # at some point instead of just randomly choosing,
    # we can use a neato estimation function for it.
    NK_pre = np.array(list(map(lambda word: 
                     KB[dic[word]] if word in dic \
                      else np.random.uniform(size=(1, 100)), batch)))
    NK = np.concatenate([NK_pre[:-1], NK_pre[1:]], axis=1)
    return NK

In [203]:
def find_X_sets(NK, KB, alpha):
    # NK is new knowledge
    # KB is our knowledge base so far
    # alpha is the deviation from the mean distance a good vector
    # to model our ideal will be
    x_ideal = NK[:, 1] - NK[:, 0].reshape(len(NK), NK.shape[-1])
    dists = vector_cos5(KB, x_ideal[0])
    return dists
    X = np.where((dists > 0) & (dists < alpha)) 
    return X

In [5]:
words = load()

In [217]:
NK = in_embed_form(["hello", "good-bye", "woops"], [], [])

KB = np.random.random_sample(size=(500, 100))

In [148]:
NK.shape

(2, 2, 100)

In [174]:
shf = np.arange(len(KB))
np.random.shuffle(shf)
s_dists = ((KB - KB[shf])**2).sum(axis=1)
alpha = (s_dists.mean(axis=0) + s_dists.std(axis=0))

In [249]:
import operator
def dot_product2(v1, v2):
    return sum(map(operator.mul, v1, v2), axis=0)


def vector_cos5(v1, v2):
    prod = dot_product2(v1, v2)
    len1 = np.sqrt(dot_product2(v1, v1))
    len2 = np.sqrt(dot_product2(v2, v2))
    return prod / (len1 * len2)


In [178]:
s_dists.std(axis=0)

2.0807058835338248

In [204]:
X = find_X_sets(NK, KB, alpha)

In [219]:
test = KB[np.random.randint(0, len(KB), size=(100))]

In [250]:
mhm = np.stack((pd.DataFrame(KB).apply(
            lambda row: vector_cos5(row, test_row).argmin(axis=0), axis=1)
          for test_row in test), axis=0)

TypeError: ('sum() takes no keyword arguments', 'occurred at index 0')

In [42]:
def complete(KB, data):
    data_c = list([data[0]])
    for index, clust in enumerate(data[:-1]):
            i = 0
            ed = KB.get_edge_data(clust, data[index+1])
            while ed and i < 20:
                data_c.append(ed['node'])
                ed = KB.get_edge_data(ed['node'], data[index+1])
                i += 1
            data_c.append(data[index+1])
    return data_c

Let $KB$ be the set of all recorded vectors, then
$$ X = \big( x \in KB \text{ s.t. } \mid dist(a,b) - x \mid \, \le \alpha \big)$$ where $\alpha$ represents the density our graph, $\epsilon$ is a noise term, and 

$$ X_S \sim N(X, \eta S + \epsilon)$$ where $S = p(S, x,y)$, then

$$ p(x\mid a,b) = 
\frac{dist^2(a,x^\ast_0) 
+ dist^2(b, x^\ast_1)}{\sum_{x\in {X_S}}dist^2(a,x_0) 
+ dist^2(b, x_1)} $$



If $ X_S $ is empty, then add $x = dist(a,b)$ to KB

In [3]:
KB = nx.read_gpickle('./KB2.gpickle')

In [13]:
embeds = np.load('./embedL2!.npy')

In [17]:
data = [KB.node[word]['number'] for word in words]

In [43]:
d_full = complete(KB, words)

In [47]:
d_full_id = [KB.node[word]['number'] for word in d_full]

In [50]:
counts = collections.Counter()
for datum in d_full_id:
    counts.update([datum])

In [52]:
revdic = {node[1]['number']:node[0] for node in KB.nodes(True)}

In [56]:
mc = [item[0] for item in counts.most_common(15)]

In [57]:
label_mc = [revdic[item] for item in mc]

In [101]:
c2 = {label: collections.Counter() for label in label_mc}

In [102]:
for index in range(len(d_full[:-1])):
    ed = KB.get_edge_data(d_full[index], d_full[index+1])
    if ed and ed['node'] in c2:
        c2[ed['node']].update([" ".join([d_full[index], d_full[index+1]])])

In [103]:
c2['20146']

Counter({',” 8715': 1,
         '9425 9623': 1,
         'As 9113': 1,
         'Before 9295': 1,
         'But 17951': 1,
         'Find 9529': 1,
         'In 9217': 1,
         'Luckily 9452': 1,
         'Miss 8700': 1,
         'Queen 9293': 2,
         'Seven 9068': 1,
         'Trims 9268': 1,
         'Turtle 19243': 12,
         'absence 8709': 1,
         'agreeable 9293': 1,
         'always 9354': 11,
         'am 8846': 3,
         'am 8859': 1,
         'apologise 8588': 1,
         'backgammon 8709': 2,
         'barrowful 8746': 1,
         'beaux 8746': 1,
         'cared 9119': 1,
         'charming 9418': 1,
         'chicken 8709': 1,
         'chiefly 8925': 1,
         'continued 9515': 1,
         'day 8643': 1,
         'done 9529': 1,
         'dwelling 9185': 1,
         'entreat 9268': 2,
         'evil 8588': 1,
         'exceedingly 8588': 1,
         'exhibit 8588': 1,
         'folly 9217': 1,
         'from 8579': 9,
         'got 9293': 2,
         'han

In [86]:
c2['20520']

Counter({82: 1,
         84: 1,
         353: 1,
         357: 1,
         383: 1,
         498: 1,
         501: 1,
         514: 2,
         582: 1,
         674: 1,
         803: 1,
         813: 1,
         866: 4,
         902: 1,
         1088: 1,
         1096: 1,
         1317: 1,
         1586: 1,
         1882: 1,
         1941: 1,
         1973: 3,
         2242: 1,
         2253: 1,
         2515: 1,
         2555: 1,
         2594: 1,
         2764: 1,
         2996: 1,
         3032: 1,
         3087: 1,
         3143: 2,
         3296: 1,
         3337: 1,
         3417: 1,
         3431: 1,
         3475: 1,
         3531: 1,
         3537: 1,
         3644: 1,
         3693: 1,
         3703: 1,
         3740: 1,
         3865: 1,
         4239: 1,
         4241: 1,
         4429: 1,
         4473: 1,
         4573: 2,
         4746: 1,
         4801: 1,
         4836: 1,
         4881: 2,
         4965: 4,
         5129: 2,
         5253: 1,
         5331: 1,
        